# `feyntrop` tutorial: A 2-loop 3-point Feynman graph

The first step is to import the `feyntrop` module, assuming `feyntrop.so` and `py_feyntrop.py` are in the working directory, we can run:

In [1]:
from py_feyntrop import *

## The graph
* We specify the Feynman graph via a list of weighted edges.
* Each item of the list is given in the format $((u,w),\nu)$, where $u$ and $w$ are the vertices connected by the edge and $\nu$ is the associated edge weight.
* We use 0-indexing. So, the vertices are numbered $0,1,2,\ldots$.
* External vertices must come first, so $V_\text{ext} = \{0,1,2\}$ label external partcles, and $V_\text{int} = \{3\}$.

The graph is then given as an edge list:

In [2]:
graph = [((0,1), 1), ((1,3), 1), ((3,2), 1), ((2,0), 1), ((0,3), 1)]

## Kinematics
* For each edge $e$ in `graph` we give an associated squared mass as list of tuples `(m_sqr[e], value)`.
* Here we choose $m_e^2 = 0.2$ for all $e$.
* For other examples where some internal masses are zero, one can use the following syntax:
    - If *no* masses are given, i.e. the `tropical_integration` function below is called *without* the last argument `masses_sqr`, then all masses are assumed to be zero.
    - If a list `masses_sqr` *is* given to `tropical_integration`, but some edge masses `(m_sqr[e], value)` are missing, then theses are asummed to be zero. For example, there might be three edges but we give `[(m_sqr[0], 1), (m_sqr[2], 2)]`, in which case `[(m_sqr[1], 0)]` is assumed.

In [3]:
masses_sqr = [(m_sqr[0], 0.2), (m_sqr[1], 0.2), (m_sqr[2], 0.2), (m_sqr[3], 0.2), (m_sqr[4], 0.2)]

* Next we provide values for the momentum configuration.
* The values for squared momenta $p_0^2, \ldots, p_{|V_\text{ext}|-2}^2$ are denoted by tuples `(p_sqr[v], value)`. In this case $|V_\text{ext}| = 3$.
* The values for Mandelstam variables $s_{ij}=(p_i+p_j)^2$ with $0 \leq i < j \leq |V_\text{ext}|-2$ are written as `(s[i,j], value)`.

In [4]:
momentum_vars = [(p_sqr[0], 0), (p_sqr[1], 0), (s[0,1], 1)]

## Additional settings
* `D0` is the integer part of the spacetime dimension $D = D_0 - 2\epsilon$.
* We expand up to but not including `eps_order`.
* `Lambda` denotes the deformation parameter $\lambda$.
* `N` is the number of Monte Carlo sampling points.

In [5]:
D0 = 2
eps_order = 5
Lambda = 7.6
N = int(1e7)

## Tropical integration
* `trop_res` is the value of the Feynman integral *without* any prefactor.
* `Itr` is the normalization factor in the tropical measure.

In [6]:
trop_res, Itr = tropical_integration(N, D0, Lambda, eps_order, graph, momentum_vars, masses_sqr)

Prefactor: gamma(2*eps + 3).
Generalized permutahedron property: fulfilled.
(Effective) kinematic regime: Minkowski
Analytic continuation: activated. Lambda = 7.6
Started integrating using 8 threads and N = 1e+07 points.
Finished in 8.32646 seconds = 0.00231291 hours.

-- eps^0: [-46.57  +/- 0.13]  +  i * [ 87.18  +/- 0.12]
-- eps^1: [-274.43 +/- 0.55]  +  i * [111.30  +/- 0.55]
-- eps^2: [-435.01 +/- 1.29]  +  i * [-174.52 +/- 1.33]
-- eps^3: [-191.38 +/- 2.14]  +  i * [-494.81 +/- 2.15]
-- eps^4: [219.81  +/- 2.68]  +  i * [-431.64 +/- 2.67]


* This yields the $\epsilon$-expansion *with* prefactor $\frac{\Gamma(\omega)}{\Gamma(\nu_1) \cdots \Gamma(\nu_E)} = \Gamma(2\epsilon+3)$ included, where $\omega$ is the superficial degree of divergence and $E$ is the number of edges.

In [7]:
eps_expansion(trop_res, graph, D0)

174.368276*I - 93.13615449 + eps*(-720.7454859 + 544.407457*I) + eps**2*(-2115.152373 + 496.4724055*I) + eps**3*(-3570.90746 - 677.8218899*I) + eps**4*(-3869.466073 - 2726.897053*I) + O(eps**5)